In [28]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [2]:
#Leo la data proveniente de un csv
path_installs = 'datos/installs.csv'
installs = pd.read_csv(path_installs)

/home/santiago/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (4,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
installs.shape

(481511, 18)

In [4]:
installs.head(10)

,created,application_id,ref_type,ref_hash,click_hash,attributed,implicit,device_countrycode,device_brand,device_model,session_user_agent,user_agent,event_uuid,kind,wifi,trans_id,ip_address,device_language
0,2019-04-24 06:23:29.495,1,1494519392962156891,4716708407362582887,NaN,False,True,6287817205707153877,NaN,3.739127e+17,adjust.com,NaN,79837499-2f2a-4605-a663-e322f759424f,app_open,NaN,NaN,4243443387795468703,3.301378e+18
1,2019-04-24 02:06:01.032,1,1494519392962156891,7143568733100935872,NaN,False,False,6287817205707153877,NaN,7.805539e+18,adjust.com,NaN,NaN,NaN,NaN,NaN,4724288679627032761,3.301378e+18
2,2019-04-20 10:15:36.274,1,1494519392962156891,5230323462636548010,NaN,False,True,6287817205707153877,NaN,8.355496e+18,adjust.com,NaN,dda99e3c-9c4b-487d-891c-79f0a02cb4a8,app_open,NaN,NaN,8291809486355890410,4.060930e+18
3,2019-04-20 21:56:47.151,1,1494519392962156891,5097163995161606833,NaN,False,True,6287817205707153877,NaN,2.355772e+18,adjust.com,NaN,7010c3ce-0fcf-46c6-9be8-374cc0e20af4,app_open,NaN,NaN,4006811922873399949,3.301378e+18
4,2019-04-20 22:40:41.239,1,1494519392962156891,6328027616411983332,NaN,False,False,6287817205707153877,NaN,6.156971e+18,adjust.com,NaN,NaN,NaN,NaN,NaN,3386455054590810771,3.301378e+18
5,2019-04-20 18:19:27.485,1,1494519392962156891,7522785771858684314,NaN,False,True,6287817205707153877,NaN,6.208879e+18,adjust.com,NaN,35e7504f-cd9b-4a78-b89f-7335a8bd735a,app_open,NaN,NaN,7297343258015606683,3.301378e+18
6,2019-04-20 03:40:21.239,1,1494519392962156891,7882044913917355073,NaN,False,True,6287817205707153877,NaN,4.566898e+18,adjust.com,NaN,f3a8649c-bd47-4874-a20e-8d3d5cedac2e,app_open,NaN,NaN,4764130939738113581,4.060930e+18
7,2019-04-23 20:55:27.691,1,1494519392962156891,404154463572960372,NaN,False,True,6287817205707153877,NaN,6.208879e+18,adjust.com,NaN,06f054c1-608b-437b-a7d7-21d7c5860fe4,app_open,NaN,NaN,5153039437575809299,3.301378e+18
8,2019-04-23 18:14:14.756,1,1494519392962156891,1448747284042458525,NaN,False,False,6287817205707153877,NaN,6.208879e+18,adjust.com,NaN,NaN,NaN,NaN,NaN,7469505621463131113,3.301378e+18
9,2019-04-23 06:32:40.906,1,1494519392962156891,1465267208368809184,NaN,False,True,6287817205707153877,NaN,4.322426e+18,adjust.com,NaN,e47c9cd6-121e-4c29-91a3-bb6d26afd36f,app_open,NaN,NaN,3744928684799347195,3.301378e+18


In [5]:
# Damos formato a la fecha.
installs['timestamp_format'] = pd.to_datetime(installs['created'])
# Agrego columna fecha
installs['date'] = installs['timestamp_format'].dt.date
# Agrego columna numero dia.
installs['day'] = installs['timestamp_format'].dt.day

In [6]:
installs.head(10)

,created,application_id,ref_type,ref_hash,click_hash,attributed,implicit,device_countrycode,device_brand,device_model,...,user_agent,event_uuid,kind,wifi,trans_id,ip_address,device_language,timestamp_format,date,day
0,2019-04-24 06:23:29.495,1,1494519392962156891,4716708407362582887,NaN,False,True,6287817205707153877,NaN,3.739127e+17,...,NaN,79837499-2f2a-4605-a663-e322f759424f,app_open,NaN,NaN,4243443387795468703,3.301378e+18,2019-04-24 06:23:29.495,2019-04-24,24
1,2019-04-24 02:06:01.032,1,1494519392962156891,7143568733100935872,NaN,False,False,6287817205707153877,NaN,7.805539e+18,...,NaN,NaN,NaN,NaN,NaN,4724288679627032761,3.301378e+18,2019-04-24 02:06:01.032,2019-04-24,24
2,2019-04-20 10:15:36.274,1,1494519392962156891,5230323462636548010,NaN,False,True,6287817205707153877,NaN,8.355496e+18,...,NaN,dda99e3c-9c4b-487d-891c-79f0a02cb4a8,app_open,NaN,NaN,8291809486355890410,4.060930e+18,2019-04-20 10:15:36.274,2019-04-20,20
3,2019-04-20 21:56:47.151,1,1494519392962156891,5097163995161606833,NaN,False,True,6287817205707153877,NaN,2.355772e+18,...,NaN,7010c3ce-0fcf-46c6-9be8-374cc0e20af4,app_open,NaN,NaN,4006811922873399949,3.301378e+18,2019-04-20 21:56:47.151,2019-04-20,20
4,2019-04-20 22:40:41.239,1,1494519392962156891,6328027616411983332,NaN,False,False,6287817205707153877,NaN,6.156971e+18,...,NaN,NaN,NaN,NaN,NaN,3386455054590810771,3.301378e+18,2019-04-20 22:40:41.239,2019-04-20,20
5,2019-04-20 18:19:27.485,1,1494519392962156891,7522785771858684314,NaN,False,True,6287817205707153877,NaN,6.208879e+18,...,NaN,35e7504f-cd9b-4a78-b89f-7335a8bd735a,app_open,NaN,NaN,7297343258015606683,3.301378e+18,2019-04-20 18:19:27.485,2019-04-20,20
6,2019-04-20 03:40:21.239,1,1494519392962156891,7882044913917355073,NaN,False,True,6287817205707153877,NaN,4.566898e+18,...,NaN,f3a8649c-bd47-4874-a20e-8d3d5cedac2e,app_open,NaN,NaN,4764130939738113581,4.060930e+18,2019-04-20 03:40:21.239,2019-04-20,20
7,2019-04-23 20:55:27.691,1,1494519392962156891,404154463572960372,NaN,False,True,6287817205707153877,NaN,6.208879e+18,...,NaN,06f054c1-608b-437b-a7d7-21d7c5860fe4,app_open,NaN,NaN,5153039437575809299,3.301378e+18,2019-04-23 20:55:27.691,2019-04-23,23
8,2019-04-23 18:14:14.756,1,1494519392962156891,1448747284042458525,NaN,False,False,6287817205707153877,NaN,6.208879e+18,...,NaN,NaN,NaN,NaN,NaN,7469505621463131113,3.301378e+18,2019-04-23 18:14:14.756,2019-04-23,23
9,2019-04-23 06:32:40.906,1,1494519392962156891,1465267208368809184,NaN,False,True,6287817205707153877,NaN,4.322426e+18,...,NaN,e47c9cd6-121e-4c29-91a3-bb6d26afd36f,app_open,NaN,NaN,3744928684799347195,3.301378e+18,2019-04-23 06:32:40.906,2019-04-23,23


In [7]:
installs18_20 = installs[(installs['day'] >= 18) & (installs['day'] < 21)]
installs19_21 = installs[(installs['day'] >= 19) & (installs['day'] < 22)]
installs20_22 = installs[(installs['day'] >= 20) & (installs['day'] < 23)]
installs21_23 = installs[(installs['day'] >= 21) & (installs['day'] < 24)]
installs22_24 = installs[(installs['day'] >= 22) & (installs['day'] < 25)]

In [8]:
installs18_20['base_time'] = datetime.datetime(2019, 4, 18)
installs18_20['time_to_install'] = (installs18_20['timestamp_format'] - installs18_20['base_time']) / np.timedelta64(1, 's')

installs19_21['base_time'] = datetime.datetime(2019, 4, 19)
installs19_21['time_to_install'] = (installs19_21['timestamp_format'] - installs19_21['base_time']) / np.timedelta64(1, 's')

installs20_22['base_time'] = datetime.datetime(2019, 4, 20)
installs20_22['time_to_install'] = (installs20_22['timestamp_format'] - installs20_22['base_time']) / np.timedelta64(1, 's')

installs21_23['base_time'] = datetime.datetime(2019, 4, 21)
installs21_23['time_to_install'] = (installs21_23['timestamp_format'] - installs21_23['base_time']) / np.timedelta64(1, 's')

installs22_24['base_time'] = datetime.datetime(2019, 4, 22)
installs22_24['time_to_install'] = (installs22_24['timestamp_format'] - installs22_24['base_time']) / np.timedelta64(1, 's')

/home/santiago/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/santiago/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/santiago/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

In [9]:
installs18_20.head(10)

,created,application_id,ref_type,ref_hash,click_hash,attributed,implicit,device_countrycode,device_brand,device_model,...,kind,wifi,trans_id,ip_address,device_language,timestamp_format,date,day,base_time,time_to_install
2,2019-04-20 10:15:36.274,1,1494519392962156891,5230323462636548010,NaN,False,True,6287817205707153877,NaN,8.355496e+18,...,app_open,NaN,NaN,8291809486355890410,4.060930e+18,2019-04-20 10:15:36.274,2019-04-20,20,2019-04-18,209736.274
3,2019-04-20 21:56:47.151,1,1494519392962156891,5097163995161606833,NaN,False,True,6287817205707153877,NaN,2.355772e+18,...,app_open,NaN,NaN,4006811922873399949,3.301378e+18,2019-04-20 21:56:47.151,2019-04-20,20,2019-04-18,251807.151
4,2019-04-20 22:40:41.239,1,1494519392962156891,6328027616411983332,NaN,False,False,6287817205707153877,NaN,6.156971e+18,...,NaN,NaN,NaN,3386455054590810771,3.301378e+18,2019-04-20 22:40:41.239,2019-04-20,20,2019-04-18,254441.239
5,2019-04-20 18:19:27.485,1,1494519392962156891,7522785771858684314,NaN,False,True,6287817205707153877,NaN,6.208879e+18,...,app_open,NaN,NaN,7297343258015606683,3.301378e+18,2019-04-20 18:19:27.485,2019-04-20,20,2019-04-18,238767.485
6,2019-04-20 03:40:21.239,1,1494519392962156891,7882044913917355073,NaN,False,True,6287817205707153877,NaN,4.566898e+18,...,app_open,NaN,NaN,4764130939738113581,4.060930e+18,2019-04-20 03:40:21.239,2019-04-20,20,2019-04-18,186021.239
12,2019-04-18 04:00:27.575,1,1494519392962156891,1084778553542198153,NaN,False,True,6287817205707153877,NaN,4.322426e+18,...,app_open,NaN,NaN,2980848185112567175,3.301378e+18,2019-04-18 04:00:27.575,2019-04-18,18,2019-04-18,14427.575
13,2019-04-18 08:33:46.588,1,1494519392962156891,6412465357609907698,NaN,False,True,6287817205707153877,NaN,7.538551e+18,...,app_open,NaN,NaN,4934642599625775274,3.301378e+18,2019-04-18 08:33:46.588,2019-04-18,18,2019-04-18,30826.588
14,2019-04-18 20:38:01.661,1,1494519392962156891,8455113036752892959,NaN,False,True,6287817205707153877,NaN,5.544094e+18,...,app_open,NaN,NaN,4312248542004529508,3.301378e+18,2019-04-18 20:38:01.661,2019-04-18,18,2019-04-18,74281.661
15,2019-04-18 18:51:18.157,1,1494519392962156891,7939167172078327528,NaN,False,False,6287817205707153877,NaN,2.355772e+18,...,NaN,NaN,NaN,8713166650905554408,4.060930e+18,2019-04-18 18:51:18.157,2019-04-18,18,2019-04-18,67878.157
22,2019-04-19 04:10:34.820,1,1494519392962156891,2251011940536423208,NaN,False,True,6287817205707153877,NaN,4.322426e+18,...,app_open,NaN,NaN,315382286036633979,3.301378e+18,2019-04-19 04:10:34.820,2019-04-19,19,2019-04-18,101434.820


In [10]:
installs18_20.describe()

,application_id,ref_type,ref_hash,device_countrycode,device_brand,device_model,ip_address,device_language,day,time_to_install
count,160861.000000,1.608610e+05,1.608610e+05,1.608610e+05,9.580300e+04,1.529780e+05,1.608610e+05,1.530420e+05,160861.000000,160861.000000
mean,111.861135,1.825809e+18,4.605664e+18,6.287817e+18,3.049725e+18,5.106135e+18,4.602120e+18,5.853168e+18,18.969035,128413.736583
std,78.096176,1.475398e+17,2.667070e+18,1.305809e+07,2.509681e+18,2.501902e+18,2.662563e+18,1.907872e+18,0.820873,76923.143543
min,1.000000,1.494519e+18,4.186353e+13,6.287817e+18,5.180696e+15,7.967346e+14,1.922017e+14,9.569587e+16,18.000000,1.560000
25%,36.000000,1.891515e+18,2.293563e+18,6.287817e+18,3.083059e+17,3.057402e+18,2.293954e+18,3.301378e+18,18.000000,68459.551000
50%,121.000000,1.891515e+18,4.603060e+18,6.287817e+18,2.208835e+18,5.274185e+18,4.586253e+18,6.977049e+18,19.000000,135385.495000
75%,155.000000,1.891515e+18,6.914026e+18,6.287817e+18,6.115026e+18,6.794880e+18,6.898795e+18,6.977049e+18,20.000000,184783.398000
max,359.000000,1.891515e+18,9.223252e+18,6.287817e+18,9.221592e+18,9.223357e+18,9.223363e+18,9.177844e+18,20.000000,259198.534000


In [11]:
installs19_21.describe()

,application_id,ref_type,ref_hash,device_countrycode,device_brand,device_model,ip_address,device_language,day,time_to_install
count,157363.000000,1.573630e+05,1.573630e+05,1.573630e+05,9.335100e+04,1.500490e+05,1.573630e+05,1.501290e+05,157363.000000,157363.000000
mean,112.104815,1.825289e+18,4.610907e+18,6.287817e+18,3.051529e+18,5.108187e+18,4.605642e+18,5.853504e+18,20.006050,131977.508433
std,78.262107,1.480058e+17,2.665592e+18,1.288708e+07,2.503270e+18,2.498361e+18,2.663174e+18,1.909063e+18,0.819085,76688.705213
min,1.000000,1.494519e+18,4.062141e+13,6.287817e+18,1.892301e+15,7.967346e+14,3.765884e+13,2.177329e+17,19.000000,0.625000
25%,36.000000,1.891515e+18,2.297844e+18,6.287817e+18,3.083059e+17,3.057402e+18,2.307150e+18,3.301378e+18,19.000000,71420.255000
50%,121.000000,1.891515e+18,4.613920e+18,6.287817e+18,2.208835e+18,5.274185e+18,4.576996e+18,6.977049e+18,20.000000,141696.007000
75%,155.000000,1.891515e+18,6.923711e+18,6.287817e+18,6.115026e+18,6.794880e+18,6.903822e+18,6.977049e+18,21.000000,187530.717500
max,359.000000,1.891515e+18,9.223252e+18,6.287817e+18,9.221592e+18,9.223357e+18,9.223350e+18,9.210226e+18,21.000000,259195.886000


In [12]:
X18_20, y18_20 = installs18_20.iloc[:,:-1],installs18_20.iloc[:,-1]

In [14]:
data_dmatrix_18_20 = xgb.DMatrix(data=X18_20,label=y18_20)

ValueError: DataFrame.dtypes for data must be int, float or bool.
                Did not expect the data types in fields created, click_hash, session_user_agent, user_agent, event_uuid, kind, wifi, trans_id, timestamp_format, date, base_time

In [15]:
X18_20.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 160861 entries, 2 to 481415
Data columns (total 22 columns):
created               160861 non-null object
application_id        160861 non-null int64
ref_type              160861 non-null int64
ref_hash              160861 non-null int64
click_hash            307 non-null object
attributed            160861 non-null bool
implicit              160861 non-null bool
device_countrycode    160861 non-null int64
device_brand          95803 non-null float64
device_model          152978 non-null float64
session_user_agent    156740 non-null object
user_agent            111880 non-null object
event_uuid            35226 non-null object
kind                  35226 non-null object
wifi                  100903 non-null object
trans_id              2867 non-null object
ip_address            160861 non-null int64
device_language       153042 non-null float64
timestamp_format      160861 non-null datetime64[ns]
date                  160861 non-null ob

In [21]:
installs18_20 = installs18_20[['application_id', 'ref_type','ref_hash', 'attributed', 'implicit', 'device_countrycode', 'device_brand', 'device_model', 'ip_address', 'device_language', 'day']]
X18_20, y18_20 = installs18_20.iloc[:,:-1],installs18_20.iloc[:,-1]

In [22]:
data_dmatrix_18_20 = xgb.DMatrix(data=X18_20,label=y18_20)

/home/santiago/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/santiago/anaconda3/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [23]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
    train_test_split(X18_20, y18_20, test_size=0.2, random_state=123)

In [24]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [25]:
xg_reg.fit(X_train,y_train)

XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.3, gamma=0, importance_type='gain',
       learning_rate=0.1, max_delta_step=0, max_depth=5,
       min_child_weight=1, missing=None, n_estimators=10, n_jobs=1,
       nthread=None, objective='reg:linear', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1)

In [26]:
preds = xg_reg.predict(X_test)

In [29]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 6.495686


In [41]:
params = {"objective":"reg:linear",'colsample_bytree': 0.3,
          'learning_rate': 0.1, 'max_depth': 5, 'alpha': 10}

cv_results = xgb.cv(dtrain=data_dmatrix_18_20, params=params, nfold=3,
                    num_boost_round=50, early_stopping_rounds=10,
                    metrics="rmse", as_pandas=True, seed=123)

In [46]:
xg_reg = xgb.train(params=params, dtrain=data_dmatrix_18_20, num_boost_round=10)

In [47]:
xgb.plot_importance(xg_reg)
plt.rcParams['figure.figsize'] = [10, 10]
plt.show()

ValueError: Booster.get_score() results in empty

In [45]:
print((cv_results["test-rmse-mean"]).tail(1))

49    0.825061
Name: test-rmse-mean, dtype: float64
